In [464]:
import pandas as pd 
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

test_id = test_data['Id']



In [465]:
train_data_y = train_data['SalePrice']
train_data_X = train_data.drop('SalePrice', axis=1)

## Feature Engineering

First---remove some features

Unneeded features: 

1. Id
2. Street
3. Alley
4. LotShape
5. LandContour
6. LotConfig
7. LandSlope
8. RoofStyle
9. RoofMatl
10. Exterior1st
11. Exterior2nd
12. MasVnrType
13. Foundation
14. Heating

In [466]:
dropped_cols = ['Id', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 
                'LandSlope', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
                'MasVnrType', 'MasVnrArea', 'Foundation', 'Heating']
train_data_X=train_data_X.drop(dropped_cols, axis=1)
test_data=test_data.drop(dropped_cols, axis=1)

## Data Cleaning

I'll start with a bit of data cleaning, checking if there's any null columns have null data, before getting more feature engineering

In [467]:
missing_X = train_data_X.isnull().sum()

In [468]:
missing_X = missing_X.sort_values()

In [469]:
temp = []
r = 0
for i in missing_X:
    if i==0:
        temp.append(missing_X.index[r])
    r+=1

print(temp)

['MSSubClass', 'LowQualFinSF', 'SaleType', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', '2ndFlrSF', 'Fireplaces', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'GarageCars', '1stFlrSF', 'GrLivArea', 'SaleCondition', 'YearBuilt', 'CentralAir', 'HouseStyle', 'BldgType', 'Condition2', 'YearRemodAdd', 'RoofStyle', 'ExterQual', 'ExterCond', 'Condition1', 'Neighborhood', 'Utilities', 'LotArea', 'BsmtFinSF1', 'MSZoning', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', 'OverallCond', 'OverallQual']


In [470]:
missing_X = missing_X.drop(temp)
missing_X

Electrical         1
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtExposure      38
BsmtFinType2      38
GarageQual        81
GarageYrBlt       81
GarageType        81
GarageCond        81
GarageFinish      81
LotFrontage      259
FireplaceQu      690
Fence           1179
MiscFeature     1406
PoolQC          1453
dtype: int64

All columns without any null values are removed in the missing_X (a Series for possible columns with null values). By instinct, most values in columns "Fence", "MiscFeature" and "PoolQC" are null. Better to delete those columns.

In [471]:
train_data_X = train_data_X.drop(['Fence', 'MiscFeature', 'PoolQC'], axis=1)

## Back to Feature Engineering

1. Fill null lotfrontage values by 0
2. one-hot encoding for other non-numeric columns


In [472]:
train_data_X['LotFrontage'].fillna(0, inplace=True)

obj_cols = train_data_X.select_dtypes(include=['object']).columns

for i in obj_cols:
    dummies = pd.get_dummies(train_data_X[i], prefix=i)
    train_data_X = pd.concat([train_data_X, dummies], axis=1)
    train_data_X.drop([i], axis=1, inplace=True)

In [473]:
train_data_X['GarageYrBlt'].fillna(0, inplace=True)

In [474]:
train_data_X

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,706,0,150,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,978,0,284,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,486,0,434,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,216,0,540,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,655,0,490,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0,0,953,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,790,163,589,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,275,0,877,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,49,1029,0,...,0,0,0,1,0,0,0,0,1,0


## Repeat these above on test data
Summarize what we've done so far on train data: 
1. Remove unnecessary columns (☑️ on test)
2. Drop columns "Fence", "MiscFeature" and "PoolQC"
3. Fill null values in LotFrontage and Garage YrBlt by 0
4. One-hot encoding on object columns
   
Repeat steps 2-4 on test data

In [475]:
test_data = test_data.drop(['Fence', 'MiscFeature', 'PoolQC'], axis=1)
test_data['LotFrontage'].fillna(0, inplace=True)
test_data['GarageYrBlt'].fillna(0, inplace=True)

for i in obj_cols:
    dummies = pd.get_dummies(test_data[i], prefix=i)
    test_data = pd.concat([test_data, dummies], axis=1)
    test_data.drop([i], axis=1, inplace=True)

By checking `train_data_X` and `test` sizes, we could see that `test` has less columns than `train_data_X`

In [476]:
set_train = set(train_data_X.columns)
set_test = set(test_data.columns)

print(set_train.difference(set_test))
print(set_test.difference(set_train))

{'GarageQual_Ex', 'Condition2_RRNn', 'Electrical_Mix', 'Utilities_NoSeWa', 'Condition2_RRAe', 'HouseStyle_2.5Fin', 'Condition2_RRAn'}
set()


Let's drop these excess columns in train set

In [477]:
diff = set_train.difference(set_test)
ld = list(diff)
train_data_X = train_data_X.drop(ld, axis=1)

In [478]:
set_train = set(train_data_X.columns)
set_test = set(test_data.columns)

print(set_train.difference(set_test))
print(set_test.difference(set_train))

set()
set()


## Finally, the prediction stage

We finally move to the prediction stage using random forest regressor

In [479]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train_data_X, train_data_y, train_size=0.9, random_state=1)

In [480]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

params = {
    "n_estimators": 2000,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
    "random_state": 1
}

model = GradientBoostingRegressor(**params)
#model = GradientBoostingRegressor(random_state=1)
model.fit(train_X, train_y)

rmse = sqrt(mean_squared_error(test_y, model.predict(test_X)))
print(rmse)

17922.427495791122


### Oops...need to clean the test data

On my first fit on this model, I found that there are null values in test data and the code has an exception for his.
Fortunately, at this stage, all columns with null data are of type int.I just fill all these columns with integers

In [481]:
test_null = test_data.isnull().sum()
temp = []
r = 0
for i in test_null:
    if i==0:
        temp.append(test_null.index[r])
    r+=1
test_null = test_null.drop(temp)
test_null


BsmtFinSF1      1
BsmtFinSF2      1
BsmtUnfSF       1
TotalBsmtSF     1
BsmtFullBath    2
BsmtHalfBath    2
GarageCars      1
GarageArea      1
dtype: int64

In [482]:
for i in test_null.index:
    test_data[i].fillna(0, inplace=True)

### Finally, the prediction

In [483]:
predict = model.predict(test_data)

In [484]:
output = pd.DataFrame({'Id': test_id.values,
                        'SalePrice': predict})

output.to_csv('submission.csv', index=False)

In [485]:
output

,Id,SalePrice
0,1461,123129.866422
1,1462,159676.078216
2,1463,182050.972207
3,1464,190935.144720
4,1465,186552.173110
...,...,...
1454,2915,84048.418969
1455,2916,73683.288678
1456,2917,167208.656880
1457,2918,123417.631019
